<a href="https://colab.research.google.com/github/mohamedhamdy9707-byte/MEDICALSEGMENTATION/blob/main/totalsegmentator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install TotalSegmentator "pyvista[jupyter]" "jupyterlab"

  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [ ]:
!gdown "https://drive.google.com/uc?export=download&id=1mgwuCpvTc3INnqGHiXhxJsbEzZPasplx" -O CT_subset_big.zip

Downloading...
From (original): https://drive.google.com/uc?export=download&id=1mgwuCpvTc3INnqGHiXhxJsbEzZPasplx
From (redirected): https://drive.google.com/uc?export=download&id=1mgwuCpvTc3INnqGHiXhxJsbEzZPasplx&confirm=t&uuid=ae92adf6-d026-4ef3-b2ab-7303b9d73fed
To: /content/CT_subset_big.zip
100% 452M/452M [00:02<00:00, 156MB/s]


In [ ]:
!unzip CT_subset_big.zip -d CT_Set

Archive:  CT_subset_big.zip
replace CT_Set/s0000/segmentations/iliopsoas_left.nii.gz? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# Segmentation


In [ ]:
from totalsegmentator.python_api import totalsegmentator
import requests
from google.colab import userdata
import os
import shutil
import zipfile
import io
import os
import nibabel as nib
import torch
import numpy as np

import torch.nn.functional as F
import ipywidgets as widgets
from IPython.display import display


# Specify the subject folder path directly
selected_folder = "/content/CT_Set/s0022" # Replace with the desired path


input_file = os.path.join(selected_folder, "ct.nii.gz")

output_dir = "output_segments"


# Delete the output directory if it exists
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)

# Create the output directory
os.makedirs(output_dir, exist_ok=True)




totalsegmentator(input_file, output_dir)

# Post-processing: Remove empty NIfTI files
for file_name in os.listdir(output_dir):
    file_path = os.path.join(output_dir, file_name)
    if file_name.endswith(".nii.gz"):
        try:
            img = nib.load(file_path)
            data = img.get_fdata()
            if np.count_nonzero(data) < 5: # Threshold for considering a mask empty
                print(f"Removing empty segmentation file: {file_name}")
                os.remove(file_path)
        except Exception as e:
            print(f"Error checking file {file_name}: {e}")

In [ ]:

import ipywidgets as widgets
from IPython.display import display
import os

# Assume output_dir is defined in a previous cell or define it here if needed
output_dir = "output_segments"

# Get list of segmentation files from the output directory
all_segmentation_files = [f for f in os.listdir(output_dir) if f.endswith(".nii.gz")]

# Extract organ names (remove .nii.gz extension)
valid_segmentation_names = [file_name.replace(".nii.gz", "") for file_name in all_segmentation_files]

# Sort the organ names alphabetically
valid_segmentation_names.sort()

# Specify the organs to visualize directly in a list
selected_organs = ["liver", "pancreas" , "kidney_right", "kidney_left" ,  "gallbladder","duodenum","portal_vein_and_splenic_vein",  ] # Replace with the list of organs you want to visualize


# # Create a dropdown widget to select organs (commented out as per user request)
# organ_selector = widgets.SelectMultiple(
#     options=valid_segmentation_names,
#     description='Select Organs:',
#     disabled=False,
#     style = {'description_width': 'initial'}
# )

# display(organ_selector)

# Now, selected_organs is a list defined in the code. The next cell will use this list.
print(f"Selected organs for visualization: { selected_organs}")

In [ ]:

import nibabel as nib
import pyvista as pv
from skimage import measure
import numpy as np
from scipy.ndimage import gaussian_filter
import colorsys
from IPython.display import display, clear_output
import ipywidgets as widgets

loaded_meshes = {}
plotter = pv.Plotter(notebook=True)
organ_controls = {}

# Load selected organ masks and create meshes
# Use the selected_organs list from the previous cell
if not selected_organs:
    print("No organs selected. Please select organs in the previous cell.")
else:
    num_organs = len(selected_organs)
    for i, organ_name in enumerate(selected_organs):
        mask_file = os.path.join(output_dir, f"{organ_name}.nii.gz")
        if os.path.exists(mask_file):
            mask = nib.load(mask_file).get_fdata()
            mask_smoothed = gaussian_filter(mask.astype(float), sigma=1)
            verts, faces, _, _ = measure.marching_cubes(mask_smoothed, level=0.5, step_size=1)
            faces = np.hstack((np.ones((faces.shape[0], 1)) * 3, faces)).flatten().astype(np.int64)
            mesh = pv.PolyData(verts, faces)
            loaded_meshes[organ_name] = mesh

            # Calculate initial color using HSV
            hue = i / num_organs  # Distribute hues evenly
            rgb_color = colorsys.hsv_to_rgb(hue, 0.8, 0.8)  # Use a fixed saturation and value
            hex_color = '#%02x%02x%02x' % (int(rgb_color[0]*255), int(rgb_color[1]*255), int(rgb_color[2]*255))

            # Add mesh to plotter and store the actor
            actor = plotter.add_mesh(mesh, color=hex_color, opacity=0.6, name=organ_name)

            # Create controls for each organ
            color_picker = widgets.ColorPicker(concise=False, description='Color:', value=hex_color, disabled=False)
            opacity_slider = widgets.FloatSlider(value=0.6, min=0.0, max=1.0, step=0.05, description='Opacity:', disabled=False, continuous_update=True, orientation='horizontal', readout=True, readout_format='.2f')
            visibility_checkbox = widgets.Checkbox(value=True, description='Visible:', disabled=False)

            # Store controls and actor
            organ_controls[organ_name] = {
                'color_picker': color_picker,
                'opacity_slider': opacity_slider,
                'visibility_checkbox': visibility_checkbox,
                'actor': actor
            }

            def update_color(change, actor=actor):
                actor.prop.color = change['new']

            def update_opacity(change, actor=actor):
                actor.prop.opacity = change['new']

            def update_visibility(change, actor=actor):
                actor.SetVisibility(change['new'])

            color_picker.observe(update_color, names='value')
            opacity_slider.observe(update_opacity, names='value')
            visibility_checkbox.observe(update_visibility, names='value')


    render_button = widgets.Button(description="Render Plot") # Moved outside the for loop

    def render(b=None): # Added b=None to accept button click event
      clear_output()
      for organ_name, controls in organ_controls.items():
          print(f"Controls for {organ_name}:")
          display(widgets.VBox([widgets.Label(value=f"{organ_name}:"), widgets.HBox([controls['color_picker'], controls['opacity_slider'], controls['visibility_checkbox']])])) # Added organ name label
      display(render_button)
      plotter.show(jupyter_backend='html')

    render_button.on_click(render)


    # Initial display of controls and plot
    render()

In [ ]:
import os
import numpy as np
import nibabel as nib
from scipy.spatial.distance import directed_hausdorff
from scipy.ndimage import distance_transform_edt
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

class SegmentationEvaluator:
    """
    Evaluates medical image segmentation using multiple metrics.
    """

    def __init__(self, prediction_dir, ground_truth_dir):
        """
        Initialize evaluator with prediction and ground truth directories.

        Args:
            prediction_dir: Directory containing predicted segmentation masks
            ground_truth_dir: Directory containing ground truth masks
        """
        self.prediction_dir = prediction_dir
        self.ground_truth_dir = ground_truth_dir
        self.results = []

    def dice_coefficient(self, pred, gt):
        """
        Calculate Dice Similarity Coefficient.

        DSC = 2 * |pred ∩ gt| / (|pred| + |gt|)

        Args:
            pred: Predicted binary mask
            gt: Ground truth binary mask

        Returns:
            float: Dice score (0-1, higher is better)
        """
        pred_bool = pred.astype(bool)
        gt_bool = gt.astype(bool)

        intersection = np.logical_and(pred_bool, gt_bool).sum()

        if pred_bool.sum() + gt_bool.sum() == 0:
            return 1.0  # Both masks are empty

        dice = (2.0 * intersection) / (pred_bool.sum() + gt_bool.sum())
        return dice

    def iou_score(self, pred, gt):
        """
        Calculate Intersection over Union (Jaccard Index).

        IoU = |pred ∩ gt| / |pred ∪ gt|

        Args:
            pred: Predicted binary mask
            gt: Ground truth binary mask

        Returns:
            float: IoU score (0-1, higher is better)
        """
        pred_bool = pred.astype(bool)
        gt_bool = gt.astype(bool)

        intersection = np.logical_and(pred_bool, gt_bool).sum()
        union = np.logical_or(pred_bool, gt_bool).sum()

        if union == 0:
            return 1.0  # Both masks are empty

        iou = intersection / union
        return iou

    def hausdorff_distance(self, pred, gt):
        """
        Calculate 95th percentile Hausdorff Distance.

        Measures the maximum surface distance between two segmentations.

        Args:
            pred: Predicted binary mask
            gt: Ground truth binary mask

        Returns:
            float: Hausdorff distance in voxels (lower is better)
        """
        pred_bool = pred.astype(bool)
        gt_bool = gt.astype(bool)

        # Check if either mask is empty
        if not pred_bool.any() or not gt_bool.any():
            return np.inf

        # Get surface points
        pred_surface = self._get_surface_points(pred_bool)
        gt_surface = self._get_surface_points(gt_bool)

        if len(pred_surface) == 0 or len(gt_surface) == 0:
            return np.inf

        # Calculate directed Hausdorff distances
        forward_distances = self._compute_surface_distances(pred_surface, gt_surface)
        backward_distances = self._compute_surface_distances(gt_surface, pred_surface)

        # Combine and take 95th percentile
        all_distances = np.concatenate([forward_distances, backward_distances])
        hausdorff_95 = np.percentile(all_distances, 95)

        return hausdorff_95

    def _get_surface_points(self, mask):
        """
        Extract surface points from a binary mask using erosion.
        """
        from scipy.ndimage import binary_erosion

        eroded = binary_erosion(mask)
        surface = mask & ~eroded
        surface_points = np.argwhere(surface)

        return surface_points

    def _compute_surface_distances(self, points1, points2):
        """
        Compute minimum distances from points1 to points2.
        """
        from scipy.spatial import cKDTree

        tree = cKDTree(points2)
        distances, _ = tree.query(points1)

        return distances

    def evaluate_organ(self, organ_name):
        """
        Evaluate a single organ segmentation.

        Args:
            organ_name: Name of the organ (e.g., 'liver', 'kidney_left')

        Returns:
            dict: Dictionary containing all metrics
        """
        pred_file = os.path.join(self.prediction_dir, f"{organ_name}.nii.gz")
        gt_file = os.path.join(self.ground_truth_dir, f"{organ_name}.nii.gz")

        if not os.path.exists(pred_file):
            print(f"Warning: Prediction file not found for {organ_name}")
            return None

        if not os.path.exists(gt_file):
            print(f"Warning: Ground truth file not found for {organ_name}")
            return None

        # Load masks
        pred_mask = nib.load(pred_file).get_fdata()
        gt_mask = nib.load(gt_file).get_fdata()

        # Binarize masks (in case they're not already binary)
        pred_mask = (pred_mask > 0).astype(np.uint8)
        gt_mask = (gt_mask > 0).astype(np.uint8)

        # Calculate metrics
        dice = self.dice_coefficient(pred_mask, gt_mask)
        iou = self.iou_score(pred_mask, gt_mask)
        hausdorff = self.hausdorff_distance(pred_mask, gt_mask)

        results = {
            'Organ': organ_name,
            'Dice Score': dice,
            'IoU': iou,
            'Hausdorff Distance (95%)': hausdorff
        }

        return results

    def evaluate_all(self, organ_list):
        """
        Evaluate all organs in the list.

        Args:
            organ_list: List of organ names to evaluate

        Returns:
            pandas.DataFrame: Results for all organs
        """
        self.results = []

        for organ in organ_list:
            print(f"Evaluating {organ}...")
            result = self.evaluate_organ(organ)
            if result is not None:
                self.results.append(result)

        df = pd.DataFrame(self.results)
        return df

    def visualize_results(self, df):
        """
        Create visualizations of the evaluation results.

        Args:
            df: DataFrame containing evaluation results
        """
        fig, axes = plt.subplots(1, 3, figsize=(15, 4))

        organs = df['Organ'].values

        # Dice Score
        axes[0].bar(organs, df['Dice Score'], color='skyblue', edgecolor='black')
        axes[0].set_ylabel('Dice Score', fontsize=12)
        axes[0].set_title('Dice Similarity Coefficient', fontsize=14, fontweight='bold')
        axes[0].set_ylim([0, 1])
        axes[0].grid(axis='y', alpha=0.3)
        axes[0].tick_params(axis='x', rotation=45)

        # IoU
        axes[1].bar(organs, df['IoU'], color='lightcoral', edgecolor='black')
        axes[1].set_ylabel('IoU Score', fontsize=12)
        axes[1].set_title('Intersection over Union', fontsize=14, fontweight='bold')
        axes[1].set_ylim([0, 1])
        axes[1].grid(axis='y', alpha=0.3)
        axes[1].tick_params(axis='x', rotation=45)

        # Hausdorff Distance
        hd_values = df['Hausdorff Distance (95%)'].replace([np.inf], np.nan)
        axes[2].bar(organs, hd_values, color='lightgreen', edgecolor='black')
        axes[2].set_ylabel('Distance (voxels)', fontsize=12)
        axes[2].set_title('Hausdorff Distance (95%)', fontsize=14, fontweight='bold')
        axes[2].grid(axis='y', alpha=0.3)
        axes[2].tick_params(axis='x', rotation=45)

        plt.tight_layout()
        plt.show()

        return fig


# ============================================
# USAGE EXAMPLE
# ============================================

# Define directories
prediction_dir = "output_segments"  # Your TotalSegmentator output
ground_truth_dir = "/content/CT_Set/s0022/segmentations"  # Ground truth directory

# Define organs to evaluate
organs_to_evaluate = ["liver", "kidney_left", "kidney_right",

                      "pancreas"]

# Create evaluator
evaluator = SegmentationEvaluator(prediction_dir, ground_truth_dir)

# Run evaluation
results_df = evaluator.evaluate_all(organs_to_evaluate)

# Display results
print("\n" + "="*60)
print("SEGMENTATION EVALUATION RESULTS")
print("="*60)
print(results_df.to_string(index=False))
print("="*60)

# Print summary statistics
print("\nSUMMARY STATISTICS:")
print("-"*60)
print(f"Average Dice Score: {results_df['Dice Score'].mean():.4f}")
print(f"Average IoU: {results_df['IoU'].mean():.4f}")

# Handle inf values in Hausdorff
hd_finite = results_df['Hausdorff Distance (95%)'].replace([np.inf], np.nan)
if not hd_finite.isna().all():
    print(f"Average Hausdorff Distance: {hd_finite.mean():.4f} voxels")
else:
    print("Average Hausdorff Distance: N/A (no valid surfaces)")

# Visualize results
evaluator.visualize_results(results_df)

# Save results to CSV
results_df.to_csv('segmentation_evaluation_results.csv', index=False)
print("\nResults saved to 'segmentation_evaluation_results.csv'")